# Needed variables

In [2]:
from getpass import getpass

CLIENT_ID = getpass("CLIENT_ID: ")
CLIENT_SECRET = getpass("CLIENT_SECRET: ")
BASE_URL = "https://dam.demo.meinbildungsraum.de/datenraum"

CLIENT_ID: ········
CLIENT_SECRET: ········


# Helper function to retrieve a new token

In [8]:
import requests

from requests.auth import HTTPBasicAuth

def get_token():
    url = "https://aai.demo.meinbildungsraum.de/realms/nbp-aai/protocol/openid-connect/token"

    response = requests.post(
        url,
        auth=HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET),
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        data={"grant_type": "client_credentials"},
    )

    assert response.status_code == 200

    return response.json()["access_token"]

# Example call to get the source for the slug `serlo`

!curl -X 'GET' \
  '{BASE_URL}/api/core/sources/slug/serlo' \
  -H 'accept: application/json' \
  -H 'Authorization: Bearer {get_token()}'

{"_links":{"self":{"href":"https://dam.demo.meinbildungsraum.de/datenraum/api/core/sources/d8f481cb-3be6-4a91-9f5b-c22b35b918ae"}},"id":"d8f481cb-3be6-4a91-9f5b-c22b35b918ae","organization":"Serlo Education e.V.","name":"Serlo Education e.V.","slug":"serlo","lastUpdated":null}

# Record in question

In [11]:
record = {
    "id": "https://serlo.org/178983",
    "name": "Aufgabe aus \"Teil A\"",
    "type": [
      "LearningResource",
      "Quiz"
    ],
    "about": [
      {
        "id": "http://w3id.org/kim/schulfaecher/s1017",
        "type": "Concept",
        "inScheme": {
          "id": "http://w3id.org/kim/schulfaecher/"
        }
      }
    ],
    "image": "https://de.serlo.org/_assets/img/meta/mathe.png",
    "creator": [
      {
        "id": "https://serlo.org/73435",
        "name": "kathongi",
        "type": "Person",
        "affiliation": {
          "id": "https://serlo.org/organization",
          "name": "Serlo Education e.V.",
          "type": "Organization"
        }
      }
    ],
    "license": {
      "id": "https://creativecommons.org/licenses/by-sa/4.0/deed.de"
    },
    "version": {
      "id": "https://serlo.org/178986"
    },
    "@context": [
      "https://w3id.org/kim/amb/context.jsonld",
      "https://schema.org",
      {
        "@language": "de"
      }
    ],
    "isPartOf": [
      {
        "id": "https://serlo.org/178171"
      }
    ],
    "publisher": [
      {
        "id": "https://serlo.org/organization",
        "name": "Serlo Education e.V.",
        "type": "Organization"
      }
    ],
    "identifier": {
      "type": "PropertyValue",
      "value": 178983,
      "propertyID": "UUID"
    },
    "inLanguage": [
      "de"
    ],
    "maintainer": {
      "id": "https://serlo.org/organization",
      "name": "Serlo Education e.V.",
      "type": "Organization"
    },
    "dateCreated": "2020-09-24T14:29:51+00:00",
    "dateModified": "2020-09-24T14:34:11+00:00",
    "isFamilyFriendly": True,
    "mainEntityOfPage": [
      {
        "id": "https://serlo.org/metadata",
        "type": "WebContent",
        "provider": {
          "id": "https://serlo.org/organization",
          "name": "Serlo Education e.V.",
          "type": "Organization"
        },
        "dateCreated": "2024-02-28T00:09:43.667Z",
        "dateModified": "2024-02-28T00:09:43.667Z"
      }
    ],
    "isAccessibleForFree": True,
    "learningResourceType": [
      {
        "id": "http://w3id.org/openeduhub/vocabs/new_lrt/a33ef73d-9210-4305-97f9-7357bbf43486"
      }
    ]
}

source_id = "d8f481cb-3be6-4a91-9f5b-c22b35b918ae"

# Get id from external id

In [13]:
from urllib.parse import urlencode

params = urlencode({ "externalId": record["id"] })

!curl -X 'GET' \
  '{BASE_URL}/api/core/nodes/external/{source_id}?{params}' \
  -H 'accept: application/json' \
  -H 'Authorization: Bearer {get_token()}'

{"id":"c6faa97c-3f62-4e45-9368-b0359f0e0fcb","title":"Aufgabe aus \"Teil A\"","description":"","externalId":"https://serlo.org/178983","sourceId":"d8f481cb-3be6-4a91-9f5b-c22b35b918ae","metadata":{"Amb":{"id":"https://serlo.org/178983","name":"Aufgabe aus \"Teil A\"","type":["LearningResource","Quiz"],"about":[{"id":"http://w3id.org/kim/schulfaecher/s1017","type":"Concept","inScheme":{"id":"http://w3id.org/kim/schulfaecher/"}}],"image":"https://de.serlo.org/_assets/img/meta/mathe.png","creator":[{"id":"https://serlo.org/73435","name":"kathongi","type":"Person","affiliation":{"id":"https://serlo.org/organization","name":"Serlo Education e.V.","type":"Organization"}}],"license":{"id":"https://creativecommons.org/licenses/by-sa/4.0/deed.de"},"version":{"id":"https://serlo.org/178986"},"@context":["https://w3id.org/kim/amb/context.jsonld","https://schema.org",{"@language":"de"}],"isPartOf":[{"id":"https://serlo.org/178171"}],"publisher":[{"id":"https://serlo.org/organization","name":"Serlo

# Update node

In [25]:
import json

record["description"] = "Example description"

data = json.dumps({
    "title": record["name"],
    "description": record["description"],
    "externalId": record["id"],
    "sourceId": source_id,
    "nodeClass": "LearningOpportunity",
    "metadata": { "Amb": record }
})

!curl -X 'PUT' \
  '{BASE_URL}/api/core/nodes/c6faa97c-3f62-4e45-9368-b0359f0e0fcb?metadataValidation=Amb' \
  -H 'accept: */*' \
  -H 'Authorization: Bearer {get_token()}' \
  -H 'Content-Type: application/json' \
  -d '{data}'